In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, Image as dsImage, Value as scalar, ClassLabel 

In [2]:
df=pd.read_feather("./data/stage3.feather")
df

,path,class,predicted,certainty
0.0,images/2006/jun/p0622063.jpg,UNKNOWN,UNK,0.998998
2.0,images/2013/sep/p0926132.jpg,UNKNOWN,UNK,0.998995
3.0,images/2019/ene/p0120193.jpg,UNKNOWN,UNK,0.998994
4.0,images/2018/jul/p0723181.jpg,UNKNOWN,UNK,0.998969
5.0,images/2015/mar/p0329153.jpg,UNKNOWN,UNK,0.998967
...,...,...,...,...
19283.0,images/2015/may/p0530155.jpg,WITH_EXPLOSION,EXP,0.489222
19284.0,images/2016/oct/p1028164.jpg,WITH_EXPLOSION,EXP,0.461512
19285.0,images/2018/dic/p1231184.jpg,WITH_EXPLOSION,EXP,0.459877
19286.0,images/2018/jun/p0603184.jpg,WITH_EXPLOSION,EXP,0.421862


In [3]:
# cleanup useless data

In [4]:
df.drop(df[(df["predicted"]=="UNK") & (df["certainty"]==0)].index, inplace=True) #cannot load these images.

In [5]:
df.drop(df[df["path"].str.contains(".ipynb")].index,inplace=True) #drop cached images from jupyter

In [6]:
df.drop(df[df["class"]=="UNKNOWN"].index, inplace=True) #drop those where there is no volcano

In [7]:
df.drop(columns=["predicted","certainty"], inplace=True) #drop the non-predicted columns

In [8]:
df.drop_duplicates(subset="path", inplace=True)

In [9]:
df["path"]=df["path"].str.replace("\\","/")

In [10]:
#normalize predictors

In [11]:
df.shape

(18481, 2)

In [12]:
#consolidate sorting

In [13]:
df.reset_index(drop=True, inplace=True)

In [14]:
df.head()

,path,class
0,images/2012/dic/p1203121.jpg,INACTIVE
1,images/2020/may/p0509201.jpg,WITH_EXPLOSION
2,images/2018/feb/p0221184.jpg,WITH_FUME_AND_EXPLOSION
3,images/2003/mar/p0312031.jpg,INACTIVE
4,images/2017/jul/p0702174.jpg,WITH_FUME


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18481 entries, 0 to 18480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    18481 non-null  object
 1   class   18481 non-null  object
dtypes: object(2)
memory usage: 288.9+ KB


In [16]:
df.shape

(18481, 2)

In [20]:
# WEIGHTS FOR CLASSIFICATION LOSS

In [41]:
df["class"].value_counts()

class
WITH_FUME                  8368
WITH_FUME_AND_EXPLOSION    7037
INACTIVE                   2574
WITH_EXPLOSION              502
Name: count, dtype: int64

In [40]:
freq=df["class"].value_counts()
freq=(df.shape[0]-freq)**4
freq=freq/freq.sum()
freq

class
WITH_FUME                  0.053332
WITH_FUME_AND_EXPLOSION    0.087454
INACTIVE                   0.326454
WITH_EXPLOSION             0.532760
Name: count, dtype: float64

In [ ]:
0.326454, 0.532760, 0.053332, 0.087454

In [18]:
df.to_feather("./data/stage4.feather")

In [19]:
def gen():
    for i, row in df.iterrows():
        yield {
            "image":row["path"],
            "class":row["class"],
        }

In [28]:
np.unique(df["class"])

array(['INACTIVE', 'WITH_EXPLOSION', 'WITH_FUME',
       'WITH_FUME_AND_EXPLOSION'], dtype=object)

In [29]:
class_names=['INACTIVE', 'WITH_EXPLOSION', 'WITH_FUME', 'WITH_FUME_AND_EXPLOSION']

In [30]:
dataset=Dataset.from_generator(gen).with_format("torch")
dataset=dataset.cast_column("image", dsImage())
dataset=dataset.cast_column("class", ClassLabel(num_classes=len(class_names), names=class_names))
dataset=dataset.train_test_split(test_size=.1, seed=36918, stratify_by_column="class")

Casting the dataset:   0%|          | 0/18481 [00:00<?, ? examples/s]

In [31]:
dataset.save_to_disk("data/dataset/",max_shard_size="80MB",num_proc=12)

Saving the dataset (0/12 shards):   0%|          | 0/16632 [00:00<?, ? examples/s]

Saving the dataset (0/12 shards):   0%|          | 0/1849 [00:00<?, ? examples/s]

In [32]:
dataset.push_to_hub("mirluvams/popocatepetl", private=True)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/8316 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/84 [00:00<?, ?ba/s]

Map:   0%|          | 0/8316 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/84 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1849 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/19 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/858 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mirluvams/popocatepetl/commit/b18e676a35cfd807267976eac356d666e5f08c5b', commit_message='Upload dataset', commit_description='', oid='b18e676a35cfd807267976eac356d666e5f08c5b', pr_url=None, pr_revision=None, pr_num=None)

# 5-MODEL TRAIN .PY

## Objetivos:

Implementación de pipeline de entrenamiento para un modelo de clasificación de imágenes utilizando `Transformers` y `PyTorch`. A continuación, te explico los pasos principales del código:

1. **Configuración del modelo**:
   - Se definen varios modelos pre-entrenados de clasificación de imágenes (ViT, Swin, etc.). El modelo seleccionado es el "google/vit-base-patch16-224", que es un Vision Transformer (ViT) de base.
   - Si el modelo no está disponible localmente, se descarga desde la biblioteca de HuggingFace.

2. **Carga del dataset**:
   - Se carga un dataset preprocesado desde el disco (`load_from_disk`).
   - El dataset es dividido en entrenamiento y prueba, con formato "torch" para su uso en PyTorch.

3. **Transformaciones de imágenes**:
   - Se aplican transformaciones a las imágenes tanto para el entrenamiento como para la prueba.
   - En el entrenamiento:
     - Las imágenes son transformadas a tipo `torch.float32`.
     - Se realizan operaciones de aumento de datos (rotación, recorte, y espejado horizontal) para evitar sobreajuste.
     - Normalización según los valores de ImageNet.
   - En la prueba:
     - Las imágenes se redimensionan y se normalizan de manera similar.

4. **Configuración del entrenamiento**:
   - Se establecen los argumentos de entrenamiento en `TrainingArguments`. Entre las opciones, se incluyen:
     - Estrategias de evaluación y guardado por épocas.
     - Uso de FP16 (precisión de punto flotante de 16 bits) para mejorar el rendimiento.
     - Definición del tamaño de lote y del número de épocas (25).
     - Métrica para seleccionar el mejor modelo: `f1` ponderado.
     - El modelo no se empujará a HuggingFace Hub en esta configuración.

5. **Collate Function**:
   - Se crea una función de "collate" personalizada, que apila los valores de píxeles y etiquetas para formar los lotes de datos.

6. **Cálculo de métricas**:
   - Se utiliza `sklearn` para calcular métricas como `accuracy` y `f1` ponderado en cada evaluación del modelo.

7. **Entrenamiento y evaluación**:
   - Se utiliza `Trainer` de HuggingFace, que simplifica el entrenamiento de modelos de aprendizaje profundo, para entrenar y evaluar el modelo.
   - Al finalizar, se guarda el mejor modelo y el estado del entrenador.